In [19]:
# Libraries
import tweepy
import json
import pandas as pd
pd.set_option('display.max_colwidth', -1) # to avoid truncated data
from pandas.io.json import json_normalize 
import datetime

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


In [2]:
!pip install langdetect
print("silence please!!!")

     |████████████████████████████████| 981 kB 1.6 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=f449d8126a9238ed5d183198e5a291d0671869d4f69646cb3097b6a248891ccb
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect
silence please!!!


In [3]:
import matplotlib.pyplot as plt
from textblob import TextBlob
import re
import numpy as np
import string

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
import nltk
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
# New dataframe columns
positive  = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

In [6]:
keyword_df = pd.read_excel("mustang_mach_e041022.xlsx")
tweets = keyword_df.full_text

In [7]:
def percentage(part,whole):
    return 100 * float(part)/float(whole)

In [8]:
noOfTweet = len(keyword_df)

for tweet in tweets:
    
    #print(tweet.text)
    tweet_list.append(tweet)
    analysis = TextBlob(tweet)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
    
    if neg > pos:
        negative_list.append(tweet)
        negative += 1

    elif pos > neg:
        positive_list.append(tweet)
        positive += 1
    
    elif pos == neg:
        neutral_list.append(tweet)
        neutral += 1

positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

In [9]:
# values from the whole data frame even duplicated data
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))

total number:  338
positive number:  151
negative number:  52
neutral number:  135


In [11]:
tweet_list.drop_duplicates(inplace = True)

In [12]:
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]
tw_list.head()

,0,text
0,@Jaswht Tesla? Mustang mach e?,@Jaswht Tesla? Mustang mach e?
1,Ford Mustang Mach E 1400 🔥\n#ForzaHorizon5 @Fo...,Ford Mustang Mach E 1400 🔥\n#ForzaHorizon5 @Fo...
4,https://t.co/F80hQJ46M8,https://t.co/F80hQJ46M8
5,@naimsharif0 Mustang Mach E,@naimsharif0 Mustang Mach E
6,@CarlKoinberg @sdmoores @benchmarkmin Yes. Mus...,@CarlKoinberg @sdmoores @benchmarkmin Yes. Mus...
...,...,...
325,101 year old area man Iived to see it.\n\nhttp...,101 year old area man Iived to see it.\n\nhttp...
326,Here's What The 2022 Ford Mustang Mach E GT Ge...,Here's What The 2022 Ford Mustang Mach E GT Ge...
328,すごい。可愛い！\nMUSTANG MACH-Eの入手方法が微塵もわかりませんけど、手に入っ...,すごい。可愛い！\nMUSTANG MACH-Eの入手方法が微塵もわかりませんけど、手に入っ...
335,Ford's Mustang Mach-E beats Tesla's Model 3 as...,Ford's Mustang Mach-E beats Tesla's Model 3 as...


In [13]:
#Creating new dataframe and new features
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]

#Removing RT, Punctuation etc
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
tw_list["text"] = tw_list.text.str.lower()
tw_list.head()

,0,text
0,@Jaswht Tesla? Mustang mach e?,tesla mustang mach e
1,Ford Mustang Mach E 1400 🔥\n#ForzaHorizon5 @Fo...,ford mustang mach e 1400 forzahorizon5
4,https://t.co/F80hQJ46M8,
5,@naimsharif0 Mustang Mach E,mustang mach e
6,@CarlKoinberg @sdmoores @benchmarkmin Yes. Mus...,yes mustang mach e battery size is 98kw...
7,@gmbutts What's your top pick right now? I hea...,what s your top pick right now i hear the i...
8,This 2022 Mustang Mach e is at the plant and i...,this 2022 mustang mach e is at the plant and i...
9,2022 Mustang Mach-E and Ford Promise Plan\n\nE...,2022 mustang mach e and ford promise plan ev ...
11,Ford Mustang Mach-E Buyers Overwhelmingly Opti...,ford mustang mach e buyers overwhelmingly opti...
12,@Kia since y’all didn’t want to reply to my FB...,since y all didn t want to reply to my fb me...


In [14]:
tw_list[['polarity', 'subjectivity']] = tw_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list['text'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        tw_list.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        tw_list.loc[index, 'sentiment'] = "positive"
    else:
        tw_list.loc[index, 'sentiment'] = "neutral"
    tw_list.loc[index, 'neg'] = neg
    tw_list.loc[index, 'neu'] = neu
    tw_list.loc[index, 'pos'] = pos
    tw_list.loc[index, 'compound'] = comp

tw_list.head(3)

,0,text,polarity,subjectivity,sentiment,neg,neu,pos,compound
0,@Jaswht Tesla? Mustang mach e?,tesla mustang mach e,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000
1,Ford Mustang Mach E 1400 🔥\n#ForzaHorizon5 @Fo...,ford mustang mach e 1400 forzahorizon5,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000
4,https://t.co/F80hQJ46M8,,0.000000,0.000000,neutral,0.000,0.000,0.000,0.0000
5,@naimsharif0 Mustang Mach E,mustang mach e,0.000000,0.000000,neutral,0.000,1.000,0.000,0.0000
6,@CarlKoinberg @sdmoores @benchmarkmin Yes. Mus...,yes mustang mach e battery size is 98kw...,-0.500000,1.000000,positive,0.000,0.803,0.197,0.4019
7,@gmbutts What's your top pick right now? I hea...,what s your top pick right now i hear the i...,0.392857,0.517857,positive,0.000,0.904,0.096,0.2023
8,This 2022 Mustang Mach e is at the plant and i...,this 2022 mustang mach e is at the plant and i...,0.400000,0.500000,positive,0.000,0.803,0.197,0.6858
9,2022 Mustang Mach-E and Ford Promise Plan\n\nE...,2022 mustang mach e and ford promise plan ev ...,0.366667,0.700000,positive,0.054,0.787,0.160,0.5719
11,Ford Mustang Mach-E Buyers Overwhelmingly Opti...,ford mustang mach e buyers overwhelmingly opti...,0.500000,1.000000,negative,0.176,0.824,0.000,-0.1280
12,@Kia since y’all didn’t want to reply to my FB...,since y all didn t want to reply to my fb me...,0.193182,0.352273,positive,0.077,0.802,0.120,0.1280


In [15]:
#Creating new data frames for all sentiments (positive, negative and neutral)

tw_list_negative = tw_list[tw_list["sentiment"]=="negative"]
tw_list_positive = tw_list[tw_list["sentiment"]=="positive"]
tw_list_neutral = tw_list[tw_list["sentiment"]=="neutral"]

In [16]:
#Function for count_values_in single columns

def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [17]:
#Count_values for sentiment
count_values_in_column(tw_list,"sentiment")

,Total,Percentage
positive,115,45.28
neutral,97,38.19
negative,42,16.54


In [21]:
tw_list_negative.head(10)

,0,text,polarity,subjectivity,sentiment,neg,neu,pos,compound
11,Ford Mustang Mach-E Buyers Overwhelmingly Opting For AWD https://t.co/hrDd0YFfaS https://t.co/IWBgWxfSVH,ford mustang mach e buyers overwhelmingly opting for awd,0.500000,1.000000,negative,0.176,0.824,0.000,-0.1280
23,"@KeepinItGutta Ye but it’s worse battery, self driving tech, and safety rating. Roughly 4 mustang Mach e of battery materials makes 5 model 3/Y… etc etc",ye but it s worse battery self driving tech and safety rating roughly 4 mustang mach e of battery materials makes 5 model 3 y etc etc,-0.250000,0.500000,negative,0.155,0.708,0.138,-0.1154
41,"@technikblog_ch @vwschweiz @AMAG_Media @cariad_tech Habe jetzt 22 Tkm mit dem Mustang Mach e und freue mich immer noch, wie am 1. Tag.\nid4 wäre die Alternative gewesen, aber es ist für mich das Pony geworden.\nIm Sommer kommt der erste elektrische Familienurlaub...freue mich drauf.\nNie mehr Verbrenner.",ch media tech habe jetzt 22 tkm mit dem mustang mach e und freue mich immer noch wie am 1 tag id4 w re die alternative gewesen aber es ist f r mich das pony geworden im sommer kommt der erste elektrische familienurlaub freue mich drauf nie mehr verbrenner,0.000000,0.000000,negative,0.083,0.917,0.000,-0.5994
48,this group of teens just hopped out a mustang mach e. i wanna be rich so bad,this group of teens just hopped out a mustang mach e i wanna be rich so bad,-0.162500,0.708333,negative,0.224,0.597,0.179,-0.2243
73,🚘 #CarReviewRating I La nueva versión más deportiva Ford Mustang Mach-E GT de 487 CV demuestra sus sorprendentes prestaciones sin emisiones en nuestra prueba https://t.co/hA8cA8q09l https://t.co/SYwAgW3Ao2,carreviewrating i la nueva versi n m s deportiva ford mustang mach e gt de 487 cv demuestra sus sorprendentes prestaciones sin emisiones en nuestra prueba,0.000000,0.000000,negative,0.146,0.769,0.085,-0.3612
98,@TheCOAPodcast @chrishibbs13 @jessicai917 @DanielTurnerPTF Finally we do not have the infrastructure to support charging these cars and also time constraints. A Kia Ev6 can quick charge in 18 minutes to 80% vs the Mustang Mach E 45 min. If you are third in line you wait a hour and still have to charge. Haven’t even touched on our grid,finally we do not have the infrastructure to support charging these cars and also time constraints a kia ev6 can quick charge in 18 minutes to 80 vs the mustang mach e 45 min if you are third in line you wait a hour and still have to charge haven t even touched on our grid,0.111111,0.500000,negative,0.074,0.879,0.047,-0.3612
100,"@wasallesbeter Een oude aftandse @FordNL mondeo 2.0 automaat uit 2002\nHij rijd nog maar hij zuipt op korte afstanden naar mijn werk!\nIk heb al een mustang mach E gevraagd aan ford, maar daar beginnen ze niet aan! Dan moet ik maar in die ouwe agenebbis auto blijven rijden!",een oude aftandse mondeo 2 0 automaat uit 2002 hij rijd nog maar hij zuipt op korte afstanden naar mijn werk ik heb al een mustang mach e gevraagd aan ford maar daar beginnen ze niet aan dan moet ik maar in die ouwe agenebbis auto blijven rijden,0.000000,0.000000,negative,0.081,0.919,0.000,-0.5994
107,New Kia EV6 – better than a Mustang Mach-E? – EAST AUTO NEWS https://t.co/lMtkvxWXfh,new kia ev6 better than a mustang mach e east auto news,0.318182,0.477273,negative,0.278,0.530,0.192,-0.3182
108,New Kia EV6 – better than a Mustang Mach-E? https://t.co/7dZqxBuQrA,new kia ev6 better than a mustang mach e,0.318182,0.477273,negative,0.347,0.413,0.240,-0.3182
113,the amount of men who have seen the bronco logo in my snaps/stories and proceeded to ask if i drive a mustang/mach e absolutely kills me every time,the amount of men who have seen the bronco logo in my snaps stories and proceeded to ask if i drive a mustang mach e absolutely kills me every time,0.200000,0.900000,negative,0.127,0.873,0.000,-0.5849
